# Lab | Chains in LangChain

## Outline

* LLMChain
* Sequential Chains
  * SimpleSequentialChain
  * SequentialChain
* Router Chain

In [1]:
# !pip install langchain

In [2]:
# !pip install langchain-openai

In [3]:
import warnings

warnings.filterwarnings('ignore')

In [4]:
import os

from dotenv import load_dotenv, find_dotenv

load_dotenv(find_dotenv('../IHAI-lessons/000_lesson_data/044_llm/.env'))

OPENAI_API_KEY = os.getenv('OPENAI_API_KEY')
HUGGINGFACEHUB_API_TOKEN = os.getenv('HF_TOKEN')

In [5]:
import pandas as pd

df = pd.read_csv('data/Data.csv')

In [ ]:
df.head()

## LLMChain

In [7]:
from langchain_openai import ChatOpenAI
from langchain.prompts import ChatPromptTemplate
from langchain.chains import LLMChain

In [8]:
# Replace None by your own value

llm = ChatOpenAI(temperature = 0.2)

In [9]:
prompt = ChatPromptTemplate.from_template("Write a funny product description for {product}. Highlight its key features and benefits. Be short and concise")

In [10]:
# old code 
# chain = LLMChain(llm=llm, prompt=prompt)

# correct code
chain = prompt | llm

In [ ]:
product = "Rubber ducks"

chain.invoke(product)

## SimpleSequentialChain

In [12]:
from langchain.chains import SimpleSequentialChain

In [13]:
llm = ChatOpenAI(temperature = 0.9)

# prompt template 1
first_prompt = ChatPromptTemplate.from_template("Write a funny product description for {product}. Highlight its key features and benefits. Be short and concise")

# Chain 1
chain_one = first_prompt | llm

In [14]:
# prompt template 2
second_prompt = ChatPromptTemplate.from_template("Based on the review {review}, rate the quality of this product from zero to ten.")

# chain 2
chain_two = second_prompt | llm

In [15]:
overall_simple_chain = chain_one | chain_two

In [ ]:
print(overall_simple_chain.invoke(product))

**Repeat the above twice for different products**

In [ ]:
# 1st product example

product_1 = df.iloc[0][0]  # Queen Size Sheet Set

print(overall_simple_chain.invoke(product_1))

In [ ]:
# 2nd product example

product_2 = df.iloc[1][0]  # Waterproof Phone Pouch

print(overall_simple_chain.invoke(product_2))

## SequentialChain

In [19]:
from langchain.chains import SequentialChain

In [ ]:
llm = ChatOpenAI(temperature = 0.9)

first_prompt = ChatPromptTemplate.from_template("Translate this {review} into french.")

chain_one = LLMChain(llm = llm, prompt = first_prompt, 
                     output_key = 'fr_review'  # Output name
                    )

In [21]:
second_prompt = ChatPromptTemplate.from_template('Based on {fr_review}, summarize this review in less than 10 words in french.')

chain_two = LLMChain(llm = llm, prompt = second_prompt, 
                     output_key = 'fr_short_review'
                    )

In [22]:
# prompt template 3: translate to english or other language
third_prompt = ChatPromptTemplate.from_template("Translate {fr_short_review} into english.")

# chain 3: input= Review and output= language
chain_three = LLMChain(llm = llm, prompt = third_prompt,
                       output_key = 'en_short_review'
                      )

In [23]:
# prompt template 4: follow up message that take as inputs the two previous prompts' variables
fourth_prompt = ChatPromptTemplate.from_template(
"Rate the quality of the translation of {fr_short_review} into {en_short_review} from zero to ten."
)

chain_four = LLMChain(llm = llm, prompt = fourth_prompt,
                      output_key = 'rating'
                     )

In [24]:
# overall_chain: input= Review 
# and output = English_Review, summary, followup_message

overall_chain = SequentialChain(
    chains = [chain_one, chain_two, chain_three, chain_four],
    input_variables = ["review"],
    output_variables = ['fr_short_review', 'en_short_review', 'rating'],
    verbose = True
)

In [ ]:
review = df.Review[5]

result = (overall_chain.invoke(review))

print(result)

**Repeat the above twice for different products or reviews**

In [ ]:
# 1st review example

product_review_1 = df['Review'][2]  # Luxury Air Mattress

result = (overall_chain.invoke(product_review_1))

print(result)

In [ ]:
# 2nd review example

product_review_2 = df['Review'][3]  # Pillows Insert

result = (overall_chain.invoke(product_review_2))

print(result)

## Router Chain

In [28]:
physics_template = """You are a very smart physics professor. 
You are great at answering questions about physics in a concise and easy to understand manner. 
When you don't know the answer to a question you admit that you don't know.

Here is a question:
{input}"""

math_template = """You are a very good mathematician. 
You are great at answering math questions. 
You are so good because you are able to break down 
hard problems into their component parts, 
answer the component parts, and then put them together 
to answer the broader question.

Here is a question:
{input}"""

history_template = """You are a very good historian. 
You have an excellent knowledge of and understanding of people, 
events and contexts from a range of historical periods. 
You have the ability to think, reflect, debate, discuss and 
evaluate the past. You have a respect for historical evidence 
and the ability to make use of it to support your explanations 
and judgements.

Here is a question:
{input}"""

computerscience_template = """ You are a successful computer scientist. 
You have a passion for creativity, collaboration, forward-thinking, 
confidence, strong problem-solving capabilities, understanding 
of theories and algorithms, and excellent communication skills. 
You are great at answering coding questions. 
You are so good because you know how to solve a problem by 
describing the solution in imperative steps that a machine 
can easily interpret and you know how to choose a solution 
that has a good balance between time complexity and space complexity. 

Here is a question:
{input}"""

In [29]:
prompt_infos = [
    {
        "name": "physics", 
        "description": "Good for answering questions about physics", 
        "prompt_template": physics_template
    },
    {
        "name": "math", 
        "description": "Good for answering math questions", 
        "prompt_template": math_template
    },
    {
        "name": "History", 
        "description": "Good for answering history questions", 
        "prompt_template": history_template
    },
    {
        "name": "computer science", 
        "description": "Good for answering computer science questions", 
        "prompt_template": computerscience_template
    }
]

prompt_templates = {
    info["name"]: ChatPromptTemplate.from_template(info["prompt_template"])
    for info in prompt_infos
}

In [30]:
from langchain.chains.router import MultiPromptChain
from langchain.chains.router.llm_router import LLMRouterChain, RouterOutputParser
from langchain.prompts import PromptTemplate

In [31]:
llm = ChatOpenAI(temperature = 0)

In [32]:
destination_chains = {}
for p_info in prompt_infos:
    name = p_info["name"]
    prompt_template = p_info["prompt_template"]
    prompt = ChatPromptTemplate.from_template(template = prompt_template)
    chain = LLMChain(llm = llm, prompt = prompt)
    destination_chains[name] = chain  
    
destinations = [f"{p['name']}: {p['description']}" for p in prompt_infos]
destinations_str = "\n".join(destinations)

In [33]:
default_prompt = ChatPromptTemplate.from_template("{input}")

default_chain = LLMChain(llm = llm, prompt = default_prompt)

In [34]:
MULTI_PROMPT_ROUTER_TEMPLATE = """Given a raw text input to a 
language model select the model prompt best suited for the input. 
You will be given the names of the available prompts and a 
description of what the prompt is best suited for. You may also 
revise the original input if you think that revising it will 
ultimately lead to a better response from the language model.

<< FORMATTING >>
Return a markdown code snippet with a JSON object formatted to look like:
```json
{{{{
    "destination": string \ name of the prompt to use or "DEFAULT"
    "next_inputs": string \ a potentially modified version of the original input
}}}}
```

REMEMBER: "destination" MUST be one of the candidate prompt names 
specified below OR it can be "DEFAULT" if the input is not well 
suited for any of the candidate prompts.
REMEMBER: "next_inputs" can just be the original input if you 
don't think any modifications are needed.

<< CANDIDATE PROMPTS >>
{destinations}

<< INPUT >>
{{input}}

<< OUTPUT (remember to include the ```json)>>"""

In [ ]:
from langchain_core.runnables import RunnableLambda, RunnablePassthrough

router_template = MULTI_PROMPT_ROUTER_TEMPLATE.format(
    destinations = destinations_str
)

router_prompt = PromptTemplate(
    template = router_template,
    input_variables = ["input"],
    output_parser = RouterOutputParser(),
)

#router_chain = router_prompt | llm | RunnableLambda(lambda x: x.content) 
router_chain = LLMRouterChain.from_llm(llm, router_prompt)

chain = MultiPromptChain(router_chain = router_chain, 
                         destination_chains = destination_chains, 
                         default_chain = default_chain, 
                         verbose = True
                        )

In [ ]:
router_chain.invoke("Explain what principle component analysis is using linear algebra")

In [ ]:
chain.invoke("What is black body radiation?")

In [ ]:
chain.invoke("what is 2 + 2")

In [39]:
# chain.run("Why does every cell in our body contain DNA?")

**Repeat the above at least once for different inputs and chains executions - Be creative!**

In [40]:
# New Example: Art Domain Router Chain

# STEP 1: Art Domain Templates

painting_template = """You are a renowned art historian specializing in painting. 
You can analyze and explain the techniques, styles, and historical context of paintings. 
When unsure, you admit your limitations.

Here is a question:
{input}"""

music_template = """You are a musicologist with expertise in musical theory, composition, and history. 
You explain concepts with clarity and connect them to broader themes in music. 
If unsure, you acknowledge your limitations.

Here is a question:
{input}"""

literature_template = """You are a literature expert with deep knowledge of authors, genres, and literary techniques. 
You provide insightful interpretations and evaluations of texts, recognizing historical and cultural contexts. 
If unsure, you admit it.

Here is a question:
{input}"""

cinema_template = """You are a film critic and historian with expertise in cinema. 
You analyze movies' artistic, technical, and cultural aspects and provide in-depth explanations. 
If unsure, you admit it.

Here is a question:
{input}"""

In [41]:
# STEP 2: Prompt Infos and Templates

prompt_infos = [
    {
        "name": "painting",
        "description": "Good for answering questions about painting",
        "prompt_template": painting_template
    },
    {
        "name": "music",
        "description": "Good for answering questions about music",
        "prompt_template": music_template
    },
    {
        "name": "literature",
        "description": "Good for answering questions about literature",
        "prompt_template": literature_template
    },
    {
        "name": "cinema",
        "description": "Good for answering questions about cinema",
        "prompt_template": cinema_template
    }
]

prompt_templates = {
    info["name"]: ChatPromptTemplate.from_template(info["prompt_template"])
    for info in prompt_infos
}

In [ ]:
# STEP 3: Destination Chains and Router Chain

from langchain.chains import LLMChain
from langchain.chains.router import MultiPromptChain
from langchain.chains.router.llm_router import LLMRouterChain, RouterOutputParser
from langchain.prompts import PromptTemplate
from langchain.chat_models import ChatOpenAI

llm = ChatOpenAI(temperature = 0)

# Create destination chains
destination_chains = {}
for p_info in prompt_infos:
    name = p_info["name"]
    prompt_template = p_info["prompt_template"]
    prompt = ChatPromptTemplate.from_template(template = prompt_template)
    chain = LLMChain(llm = llm, prompt = prompt)
    destination_chains[name] = chain  

# Define destinations and router prompt
destinations = [f"{p['name']}: {p['description']}" for p in prompt_infos]
destinations_str = "\n".join(destinations)

default_prompt = ChatPromptTemplate.from_template("{input}")
default_chain = LLMChain(llm = llm, prompt = default_prompt)

MULTI_PROMPT_ROUTER_TEMPLATE = """Given a raw text input to a 
language model, select the model prompt best suited for the input. 
You will be given the names of the available prompts and a 
description of what the prompt is best suited for. You may also 
revise the original input if you think that revising it will 
ultimately lead to a better response from the language model.

<< FORMATTING >>
Return a markdown code snippet with a JSON object formatted to look like:
```json
{{{{
    "destination": string \ name of the prompt to use or "DEFAULT"
    "next_inputs": string \ a potentially modified version of the original input
}}}}
```

REMEMBER: "destination" MUST be one of the candidate prompt names 
specified below OR it can be "DEFAULT" if the input is not well 
suited for any of the candidate prompts.
REMEMBER: "next_inputs" can just be the original input if you 
don't think any modifications are needed.

<< CANDIDATE PROMPTS >>
{destinations}

<< INPUT >>
{{input}}

<< OUTPUT (remember to include the ```json)>>"""

In [43]:
# STEP 4: Combine everything into a MultiPromptChain

router_prompt = PromptTemplate(
    template = MULTI_PROMPT_ROUTER_TEMPLATE.format(
        destinations = destinations_str), 
        input_variables = ["input"], 
        output_parser = RouterOutputParser())

router_chain = LLMRouterChain.from_llm(llm, router_prompt)

chain = MultiPromptChain(
    router_chain = router_chain, 
    destination_chains = destination_chains, 
    default_chain = default_chain, 
    verbose = True)

In [ ]:
# STEP 5: Different Inputs

### Painting Query ###
chain.invoke("What techniques did Van Gogh use in his paintings?")

In [ ]:
### Music Query ###
chain.invoke("Can you explain the concept of counterpoint in classical music?")

In [ ]:
### Default Query ### (not working - not using default)
chain.invoke("What is the relationship between art and technology?")